### Note book Two

Compare the survey results for specific items indentified during shoreline surveillance of Lake Geneva. Group survey results in time by Winter, Spring, Summer or Fall, by body of water and municipality.

__Research question:__

_Does the data indicate that shoreline litter densities are seasonal? If so describe the relationship._

#### Contents

1. Automating some things
2. Making a MlwCode class  
3. Dealing with time
4. Grouping by season
5. Plot grouped by season and code


### Setting up

Make sure you are running the same packages as this notebook. There is a requirements.txt file for the virutal environment in the [repo](https://github.com/hammerdirt/SWE_2019.git). 

#### Imports

No changes form note book one

In [1]:
import numpy as np
import json
import csv
import datetime
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
import os
import matplotlib.dates as mdates
from matplotlib import colors as mcolors
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [2]:
# making the file structure
# if you changed the file structure then this function will be problematic


folders = ["Data", "Charts", "Utilities"]
here = os.getcwd()
# never comment this out -- it is used to save output
def make_folders():
    my_folders = {}
    for folder in folders:
        place = here +"/"+ folder
        my_folders[folder] = place
    return my_folders
my_folders = make_folders()

### Making the code class

Gather and compute details for the MLW Codes in a class. 

#### Utility functions

In [3]:
# nothing fancy, jsut storing the data we used in Notebook one in a class object.

# variables
variable_names=["beach_info", "get_toys", "get_sheeting"]
end_points = [
    "http://mwshovel.pythonanywhere.com/dirt/beaches/Lac-L%C3%A9man/",
    "http://mwshovel.pythonanywhere.com/dirt/codes/Lac-L%C3%A9man/G32",
    "http://mwshovel.pythonanywhere.com/dirt/codes/Lac-L%C3%A9man/G67",    
]

def get_the_data(variable_names,end_points):
    data = {}
    for i, name in enumerate(variable_names):
        data[name] = requests.get(end_points[i]).json()
    return data
# make a pieces per meter value for each observation
def get_pieces_per_meter(a_list_of_objects):
    new_list_of_objects = []
    the_keys = list(a_list_of_objects.keys())    
    for this_key in the_keys:
        new_list = a_list_of_objects[this_key]
        for the_object in new_list:
            new_object = the_object
            new_object["pcs_m"] = np.round(new_object["quantity"]/new_object["length"], 3)
            new_list_of_objects.append(new_object)
    return new_list_of_objects
# make a summary of the data 
def make_summary(data):
    results = {}
    pcs_m = []
    for result in data:
        pcs_m.append(result["pcs_m"])
    results["the_min"] = np.min(pcs_m)
    results["the_max"] = np.max(pcs_m)
    results["the_median"] = np.median(pcs_m)
    results["the_average"] = np.mean(pcs_m)
    results["twenty_fifth"] = np.percentile(pcs_m, 25)
    results["seventy_fifth"] =  np.percentile(pcs_m, 75)
    results["no_samples"] = len(pcs_m)
    results["last_sample"] = max([x["date"] for x in data])
    results["first_sample"] = min([x["date"] for x in data])
    return results
# make time value pairts
def time_value_pairs(aList):
    myPairs = [[result['date'], result['pcs_m']] for result in aList]
    return myPairs  
    

#### Wrting the class

In [4]:
code = "G32"
common_name = ["toys_data"]
end_point = ["http://mwshovel.pythonanywhere.com/dirt/codes/Lac-L%C3%A9man/" + code]

class MlWCodes():
    def __init__(self, code, common_names, end_points):
        self.code = code
        self.data = get_pieces_per_meter(get_the_data(common_names,end_points))
        self.time_value = time_value_pairs(self.data)
        self.summary = make_summary(self.data)
        
    
toys = MlWCodes(code=code, common_names=common_name, end_points=end_point)     

#### Using the class

In [5]:
# get the summary of data for toys
toys.summary

{'the_min': 0.012,
 'the_max': 2.222,
 'the_median': 0.063,
 'the_average': 0.12757894736842101,
 'twenty_fifth': 0.027750000000000004,
 'seventy_fifth': 0.125,
 'no_samples': 76,
 'last_sample': '2019-10-10',
 'first_sample': '2015-11-23'}

### Searching, grouping and sorting by date or date range

1. Get the results for one survey day at a location
2. Group survey results by date range

The MLWCodes class includes an array of results in pcs/m per survey __MLWCodes.time_value__ or the pieces per survey is in __MLWCodes.data__.

First thing is to get the latest sample by the SWE group and take a look at those results.

####  Get the results for one survey from the data

1. Specify date and location
2. Compare the results of that sample to the sample population

In [6]:
# call the function to get data 
all_the_data = get_the_data(["pierrettes"], ["http://mwshovel.pythonanywhere.com/dirt/codes/Parc-des-Pierrettes"])

# define a method that accpets the data, a key and and the required date in integer form
def get_the_latest(all_the_data, key, year, month, day):
    the_results = all_the_data[key]
    my_date_string = '{year}-{month}-{day}'.format(year=year, month=month, day=day)
    i_want_these = [item for item in the_results if item['date'] == my_date_string]
    return i_want_these
    
the_latest_inventory = get_the_latest(all_the_data, "pierrettes", 2019,10,10)

##### get the total number found, the pieces per meter and the number of categories

In [7]:
def get_some_stats(aList):
    the_total = 0
    pcs_m = 0
    the_most = 0
    the_most_object = ''
    number_of_categories = len(aList)
    for result in aList:
        the_total += result["quantity"]
        pcs_m += result['quantity']/result['length']
        if result["quantity"] > the_most:
            the_most = result['quantity']
            the_most_object = result['code_id']
    return {'pcs-m':pcs_m, 'the_total': the_total, 'categories':number_of_categories, 'the_max':the_most, 'the_max_object':the_most_object}
get_some_stats(the_latest_inventory)

{'pcs-m': 6.366666666666665,
 'the_total': 191,
 'categories': 28,
 'the_max': 70,
 'the_max_object': 'G82'}

##### Research questions:

1. How does the pcs_m value found on that day compare to the results for the whole lake?
2. What was the probability of finding that value? (given past results)
3. Does the_max_object fall in the top ten of the whole lake?
4. How does the value for pcs-m toys compare to the previous results?

##### __Given the previous results, do you feel the latest survey results were predictable? How would you go about predicting values in the future?__


#### Group the results by month

1. Get the median, average, 50th percentile, 75th percentile and number of samples per month
2. Identify which, if any, may have a seasonal variation
3. Determine the method to verify seasonal variation

In [8]:
# check the first entries in toys.value
# it gives a date and a value in pcs/m
print(toys.time_value[0])

['2015-11-23', 0.066]


In [9]:
# convert all string values to datetime objects
toys_dates_datetime = [[datetime.datetime.strptime(x[0], "%Y-%m-%d"), x[1]] for x in toys.time_value]
toys_dates_datetime[0]

[datetime.datetime(2015, 11, 23, 0, 0), 0.066]

In [10]:
#  group by month
def group_values_by_month(aList):
    grouped_by_month = {}
    for x in aList:
        month = x[0].month
        months = grouped_by_month.keys()
        if month in months:
            grouped_by_month[month].append(x)
        else:
            grouped_by_month[month] = [x]
    return grouped_by_month
toys_grouped_by_month = group_values_by_month(toys_dates_datetime)

In [11]:
# group by month and year

def group_values_by_month_year(aList):
    grouped_by_month_year = {}
    for x in aList:
        month_year = (x[0].year, x[0].month)
        months_years = grouped_by_month_year.keys()
        if month_year in months_years:
            grouped_by_month_year[month_year].append(x)
        else:
            grouped_by_month_year[month_year] = [x]
    return grouped_by_month_year
toys_grouped_by_month_year = group_values_by_month_year(toys_dates_datetime)